# 测试流程示例

演示如何为 Xenium 与 Visium 数据计算基因程序分数、生成真值宽表，并导出解卷积输入。


## 目录
- 环境配置与基因集准备
- Xenium 基因程序打分
- Xenium 真值生成
- Visium 基因程序打分
- 数据对齐与 NPZ 导出


## 1. 环境配置与基因集设定

导入所需依赖，注册包路径，并定义用于测试的 4 组基因程序。


In [1]:
"""
预处理管线（可脚本运行）：
  - Xenium/Visium 读入、基因集打分
  - Xenium 真值计算（按 spot × broad_annotation 分组均值）
  - Visium 与细胞比例矩阵对齐，导出标准 npz（供模型使用）

注意：本版本移除了绝对路径与 os.chdir，改为 argparse 参数化；
      依赖的 compute_truth_score 当前含交互式清洗，下一步将改为无交互参数。
"""
import os
import sys
import argparse
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc

os.chdir('/home/vs_theg/ST_program')
# -------------------------------------------------------------------------
# 1. 添加模块路径并导入函数
# -------------------------------------------------------------------------
sys.path.append("/home/vs_theg/ST_program/CellType_GP/src/")

from score_gene_program import score_gene_programs
from compute_truth_score import compute_truth_score

# -------------------------------------------------------------------------
# 2. 定义 13 个 celltype Clusters 的基因集
# -------------------------------------------------------------------------
B_cell_genes = [
    'TENT5C', 'SEC11C', 'TIFA', 'MZB1', 'CD79A', 'CD19', 'MS4A1', 'TNFRSF17', 'CCPG1', 'BANK1', 
    'CD79B', 'ITM2C', 'SLAMF7', 'CD27', 'RAB30', 'PRDM1'
]

T_cell_genes = [
    'FOXP3', 'CCL20', 'IL7R', 'CTLA4', 'SLAMF1', 'KLRB1', 'LTB', 'TCF7', 'DUSP2', 'GNLY', 
    'KLRC1', 'CD8A', 'KLRD1', 'NKG7', 'PDCD1', 'CD8B', 'CCL5', 'GZMA', 'PRF1', 'CD3G', 
    'CD247', 'KLRF1', 'CD3E', 'CD3D', 'TIGIT', 'TRAC', 'APOBEC3B', 'GZMK', 'CYTIP', 'IL2RG', 
    'PTPRC', 'ADGRE5'
]

DCIS_Tumor_genes = [
    'CD9', 'QARS', 'GATA3', 'LYPD3', 'C6orf132', 'KLF5', 'SEC24A', 'TFAP2A', 'HPX', 'SMS', 
    'TPD52', 'SDC4', 'FLNB', 'HOOK2', 'TRAPPC3', 'CLDN4', 'DNTTIP1', 'CTH', 'CEACAM8', 'ESR1', 
    'CEACAM6', 'SERPINA3', 'KRT8', 'S100A14', 'AGR3', 'AQP3'
]

Endothelial_genes = [
    'RAPGEF3', 'WARS', 'HOXD8', 'EGFL7', 'PPARG', 'SOX18', 'ESM1', 'KDR', 'VWF', 'NOSTRIN', 
    'SOX17', 'RAMP2', 'CLEC14A', 'HOXD9', 'MMRN2', 'SNAI1', 'BTNL9', 'TCF15', 'CLDN5', 'AQP1', 
    'EDN1', 'CD93', 'CAVIN2', 'ANGPT2', 'BACE2', 'PECAM1', 'CAV1', 'EDNRB', 'TCF4', 'AKR1C3', 
    'ZEB1', 'LDHB'
]

Invasive_Tumor_genes = [
    'ABCC11', 'SERHL2', 'TCIM', 'FASN', 'AR', 'PTRHD1', 'TRAF4', 'USP53', 'SCD', 'SQLE', 
    'MYO5B', 'DNAAF1', 'FOXA1', 'EPCAM', 'CTTN', 'MLPH', 'ELF3', 'ANKRD30A', 'ENAH', 'KARS', 
    'CCND1', 'CCDC6', 'ERBB2', 'TMEM147', 'MDM2', 'NARS', 'OCIAD2', 'KRT7', 'LARS', 'NPM3', 
    'ZNF562', 'POLR2J3', 'HMGA1', 'C2orf42', 'THAP2', 'SH3YL1', 'TOMM7', 'CDH1', 'EIF4EBP1', 'REXO4', 
    'STC1', 'RHOH', 'SRPK1', 'DAPK3', 'SLC4A1', 'PCLAF', 'ANKRD28', 'CENPF', 'MKI67', 'TOP2A', 
    'RTKN2', 'TUBA4A', 'LAG3'
]

IRF7_plus_DCs_genes = [
    'LILRA4', 'TCL1A', 'PLD4', 'IL3RA', 'GZMB', 'DERL3', 'SPIB', 'ERN1', 'PTGDS', 'SELL', 
    'CD4', 'CLECL1', 'CXCR4', 'GPR183', 'GLIPR1', 'LPXN'
]

LAMP3_plus_DCs_genes = [
    'BASP1', 'VOPP1', 'PELI1', 'CD83', 'DUSP5', 'CD80', 'CCR7', 'CRHBP', 'CD274', 'PDCD1LG2', 
    'MAP3K8', 'FAM49A', 'SERPINB9'
]

Macrophages_genes = [
    'APOBEC3A', 'APOC1', 'MMP12', 'ITGAX', 'HAVCR2', 'FCGR3A', 'CD1C', 'IGSF6', 'LY86', 'FGL2', 
    'CX3CR1', 'LYZ', 'CLEC9A', 'C15orf48', 'FCER1A', 'CD86', 'MNDA', 'MPO', 'MRC1', 'CD14', 
    'CD163', 'C1QA', 'C1QC', 'CD68', 'PDK4', 'ITGAM', 'FCER1G', 'IL2RA', 'AIF1', 'TYROBP', 
    'CCL8', 'SMAP2'
]

Mast_genes = [
    'HDC', 'CYP1A1', 'CTSG', 'CPA3', 'TPSAB1', 'LIF', 'KIT', 'CD69', 'PDE4A', 'FAM107B'
]

Myoepi_ACTA2_plus_genes = [
    'DMKN', 'DSP', 'DST', 'OXTR', 'ACTG2', 'CLCA2', 'RUNX1', 'KRT14', 'KRT5', 'MMP1', 
    'EGFR', 'MYLK', 'PGR'
]

Myoepi_KRT15_plus_genes = [
    'RORC', 'ADAM9', 'SLC5A6', 'JUP', 'TUBB2B', 'TACSTD2', 'TRIB1', 'SLC25A37', 'SVIL', 'DSC2', 
    'ELF5', 'CXCL16', 'CDC42EP1', 'MYBPC1', 'PIGR', 'KRT23', 'SCGB2A1', 'KRT15', 'CXCL5', 'OPRPN', 
    'S100A8', 'C5orf46', 'ALDH1A3', 'KRT16', 'KRT6B', 'SFRP1', 'PTN', 'NCAM1'
]

Perivascular_Like_genes = [
    'LGALSL', 'FSTL3', 'NDUFA4L2', 'ANKRD29', 'AHSP', 'AVPR1A', 'ACTA2', 'MYH11', 'FOXC2', 'PDGFRB', 
    'SSTR2', 'FBLIM1', 'ZEB2', 'S100A4'
]

Stromal_genes = [
    'MUC6', 'TAC1', 'UCP1', 'LEP', 'CRISPLD2', 'PCOLCE', 'GJB2', 'ADIPOQ', 'CSF3', 'LRRC15', 
    'MMP2', 'FBLN1', 'POSTN', 'PDGFRA', 'TIMP4', 'IGF1', 'SFRP4', 'MEDAG', 'CCDC80', 'LUM', 
    'ADH1B', 'TCEAL7', 'LPL', 'DPT', 'CXCL12', 'AKR1C1', 'CLIC6', 'PIM1'
]


# 合并所有基因集用于评分
gene_sets_to_score = {
    'B_cell_score': B_cell_genes,
    'T_cell_score': T_cell_genes,
    'DCIS_Tumor_score': DCIS_Tumor_genes,
    'Endothelial_score': Endothelial_genes,
    'Invasive_Tumor_score': Invasive_Tumor_genes,
    'IRF7_plus_DCs_score': IRF7_plus_DCs_genes,
    'LAMP3_plus_DCs_score': LAMP3_plus_DCs_genes,
    'Macrophages_score': Macrophages_genes,
    'Mast_score': Mast_genes,
    'Myoepi_ACTA2_plus_score': Myoepi_ACTA2_plus_genes,
    'Myoepi_KRT15_plus_score': Myoepi_KRT15_plus_genes,
    'Perivascular_Like_score': Perivascular_Like_genes,
    'Stromal_score': Stromal_genes,
}
def main():
    """
    主函数：设置参数、检查文件，并返回配置。
    """
    parser = argparse.ArgumentParser(description="Preprocess ST data: scoring + truth + aligned NPZ")
    
    # --- 1. 定义参数 ---
    # 使用 pathlib.Path 作为类型，使其成为可调用的路径构造函数
    # 默认路径是相对于项目根目录的相对路径
    parser.add_argument("--xenium", type=Path, default=Path("CellType_GP/data/xdata.h5"), 
                        help="Xenium AnnData (.h5ad/.h5) path")
    parser.add_argument("--visium", type=Path, default=Path("CellType_GP/data/vdata.h5"), 
                        help="Visium AnnData (.h5ad/.h5) path")
    parser.add_argument("--fractions", type=Path, default=Path("CellType_GP/data/spot_cluster_fraction_matrix.csv"), 
                        help="Spot-celltype fractions CSV (index=spot)")
    parser.add_argument("--x2v-map", type=Path, default=Path("CellType_GP/data/xenium_to_visium_transcript_mapping.csv"), 
                        help="Xenium->Visium barcode mapping CSV")
    parser.add_argument("--outdir", type=Path, default=Path("CellType_GP/data/preprocessing_outputs"), 
                        help="Output directory for processed files")

    # --- 2. 解析参数 ---
    # 在 Notebook 中使用 parse_args([]) 是一个很好的实践，
    # 它强制 argparse 使用在 add_argument 中定义的 `default` 值，
    # 而不是去读取（不存在的）命令行输入。
    args = parser.parse_args([]) 
    
    # --- 3. 创建输出目录 ---
    outdir = args.outdir
    outdir.mkdir(parents=True, exist_ok=True)
    
    print(f"Output directory set to: {outdir.absolute()}")

    # --- 4. 检查关键文件是否存在 ---
    print("Checking file paths...")
    key_files_to_check = {
        "xenium": args.xenium,
        "visium": args.visium,
        "fractions": args.fractions,
        "x2v-map": args.x2v_map
    }

    for name, path in key_files_to_check.items():
        exists = path.exists()
        print(f"  - {name}: {path} ... {'✅ Exists' if exists else '❌ NOT FOUND'}")
        
        # 如果关键文件（如此处的 xenium）缺失，则应立即报错
        if not exists and name == "xenium": 
            raise FileNotFoundError(
                f"Key file '{name}' missing from expected path: {path.absolute()}\n"
                "Please ensure your data is correctly placed in CellType_GP/data/"
            )

    # --- 5. 返回配置 ---
    # （原始代码中此处的重复代码块已被移除）
    
    print(f"Configuration loaded successfully. Using Xenium data from: {args.xenium.absolute()}")
    
    # 返回 args 和 outdir，使其在 Notebook 的全局作用域中可用
    return args, outdir


/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
print("Running setup...")
# 调用 main() 并捕获返回值，将其存为全局变量
try:
    args, outdir = main()
except FileNotFoundError as e:
    print(e)
    # 如果文件未找到，你可能希望在这里处理错误，而不是让后续单元格失败
except Exception as e:
    print(f"An unexpected error occurred during setup: {e}")
    
print("Setup complete.")

Running setup...
Output directory set to: /home/vs_theg/ST_program/CellType_GP/data/preprocessing_outputs
Checking file paths...
  - xenium: CellType_GP/data/xdata.h5 ... ✅ Exists
  - visium: CellType_GP/data/vdata.h5 ... ✅ Exists
  - fractions: CellType_GP/data/spot_cluster_fraction_matrix.csv ... ✅ Exists
  - x2v-map: CellType_GP/data/xenium_to_visium_transcript_mapping.csv ... ✅ Exists
Configuration loaded successfully. Using Xenium data from: /home/vs_theg/ST_program/CellType_GP/data/xdata.h5
Setup complete.


## 2. Xenium 基因程序打分

在 Xenium 单细胞矩阵上运行 `score_gene_programs`，并把结果落盘。


In [3]:
# ---------------------------------------------------------------------
# 3. Xenium 数据打分
# ---------------------------------------------------------------------
adata_x = sc.read(str(args.xenium))
score_gene_programs(adata_x, gene_sets_to_score, platform="xenium", output_dir=str(outdir / "xenium_scores"))
adata_x.write(str(outdir / "xdata_processed.h5"))

🔹 正在处理: B_cell_score
✅ 完成: B_cell_score (16 基因)

🔹 正在处理: T_cell_score
✅ 完成: T_cell_score (32 基因)

🔹 正在处理: DCIS_Tumor_score
✅ 完成: DCIS_Tumor_score (26 基因)

🔹 正在处理: Endothelial_score
✅ 完成: Endothelial_score (32 基因)

🔹 正在处理: Invasive_Tumor_score
✅ 完成: Invasive_Tumor_score (53 基因)

🔹 正在处理: IRF7_plus_DCs_score
✅ 完成: IRF7_plus_DCs_score (16 基因)

🔹 正在处理: LAMP3_plus_DCs_score
✅ 完成: LAMP3_plus_DCs_score (13 基因)

🔹 正在处理: Macrophages_score
✅ 完成: Macrophages_score (32 基因)

🔹 正在处理: Mast_score
✅ 完成: Mast_score (10 基因)

🔹 正在处理: Myoepi_ACTA2_plus_score
✅ 完成: Myoepi_ACTA2_plus_score (13 基因)

🔹 正在处理: Myoepi_KRT15_plus_score
✅ 完成: Myoepi_KRT15_plus_score (28 基因)

🔹 正在处理: Perivascular_Like_score
✅ 完成: Perivascular_Like_score (14 基因)

🔹 正在处理: Stromal_score
✅ 完成: Stromal_score (28 基因)


🎉 所有基因集打分完成（使用平均表达），结果已保存至 CellType_GP/data/preprocessing_outputs/xenium_scores/



In [4]:
adata_x.obs

,cell_id,total_counts,Barcode,Cluster,broad_annotation,B_cell_score,T_cell_score,DCIS_Tumor_score,Endothelial_score,Invasive_Tumor_score,IRF7_plus_DCs_score,LAMP3_plus_DCs_score,Macrophages_score,Mast_score,Myoepi_ACTA2_plus_score,Myoepi_KRT15_plus_score,Perivascular_Like_score,Stromal_score
0,1,29,1,DCIS_2,Tumor,0.000000,0.000000,0.145091,0.033544,0.178392,0.000000,0.000000,0.000000,0.107339,0.082569,0.053001,0.000000,0.000000
1,2,94,2,DCIS_2,Tumor,0.043623,0.021811,0.343471,0.000000,0.350866,0.000000,0.053690,0.000000,0.000000,0.124226,0.099234,0.000000,0.000000
2,4,11,4,Invasive_Tumor,Tumor,0.082891,0.000000,0.000000,0.000000,0.150142,0.098333,0.000000,0.000000,0.132625,0.000000,0.000000,0.000000,0.000000
3,5,48,5,DCIS_2,Tumor,0.056902,0.000000,0.315546,0.000000,0.255753,0.000000,0.070033,0.120287,0.000000,0.000000,0.104955,0.065031,0.043920
4,8,39,8,DCIS_2,Tumor,0.080253,0.000000,0.232625,0.042373,0.190605,0.000000,0.074956,0.060902,0.097443,0.000000,0.034801,0.069602,0.069602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135087,166496,379,166496,Stromal,Stromal,0.057595,0.083349,0.238075,0.155119,0.229912,0.175207,0.198933,0.245950,0.170439,0.351025,0.195887,0.274165,0.344487
135088,166497,186,166497,Stromal,Stromal,0.118759,0.255826,0.270991,0.091860,0.231229,0.276140,0.037686,0.188232,0.168119,0.129323,0.095037,0.205176,0.287799
135089,166498,255,166498,Stromal,Stromal,0.163665,0.081832,0.225512,0.146281,0.201638,0.109656,0.186093,0.195792,0.040320,0.226562,0.079652,0.210379,0.438149
135090,166499,331,166499,Stromal,Stromal,0.096159,0.132152,0.156274,0.086343,0.152865,0.328422,0.274777,0.368599,0.067643,0.204395,0.148178,0.158212,0.302224


## 3. 生成 Xenium 真值宽表

合并 Xenium 注释后计算聚合得分，输出到 `truth_output/` 目录。


In [5]:
# ---------------------------------------------------------------------
# 4. Xenium 真值计算（合并条形码映射；清洗后按 group 平均）
# ---------------------------------------------------------------------
x2v = pd.read_csv(args.x2v_map)
# 自动检测 Xenium 细胞 ID 列 / auto-detect xenium cell id column
possible_keys = ["cell_id", "Barcode", "cell_ID", "xenium_cell_id"]
left_key = next((k for k in possible_keys if k in adata_x.obs.columns), None)
if left_key is None:
    raise ValueError("❌ 在 adata_x.obs 中找不到匹配的 cell id 列（尝试列：" + ", ".join(possible_keys) + ")")

# 合并到 obs 上（左连接），保留 Visium 条形码 / merge mapping into obs
adata_x.obs = adata_x.obs.merge(
    x2v[["xenium_cell_id", "transcript_level_visium_barcode"]],
    how="left",
    left_on=left_key,
    right_on="xenium_cell_id"
)
if "xenium_cell_id" in adata_x.obs.columns:
    adata_x.obs.drop(columns=["xenium_cell_id"], inplace=True)

before = adata_x.n_obs
adata_x = adata_x[adata_x.obs["transcript_level_visium_barcode"].notna()].copy()
after = adata_x.n_obs
print(f"✅ 去除无 transcript_level_visium_barcode 的细胞：删除 {before - after}，剩余 {after}")

# 计算真值（注意：当前 compute_truth_score 含交互式清洗，下一步将去交互化）
compute_truth_score(
    adata_x,
    output_dir=str(outdir / "truth_output"),
    spot_col='transcript_level_visium_barcode',
    celltype_col='broad_annotation',
    drop_columns=[]
)


/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/vs_theg/ST_program/CellType_GP/src/compute_truth_score.py:92: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  truth_wide = truth_result.pivot_table(index=spot_col, columns=celltype_col, values=program_cols)


✅ 去除无 transcript_level_visium_barcode 的细胞：删除 0，剩余 135092
🚀 开始计算 Xenium truth score, 输出路径: CellType_GP/data/preprocessing_outputs/truth_output

🧾 当前 adata.obs 列如下（可直接复制粘贴）：

cell_id, total_counts, Barcode, Cluster, broad_annotation, B_cell_score, T_cell_score, DCIS_Tumor_score, Endothelial_score, Invasive_Tumor_score, IRF7_plus_DCs_score, LAMP3_plus_DCs_score, Macrophages_score, Mast_score, Myoepi_ACTA2_plus_score, Myoepi_KRT15_plus_score, Perivascular_Like_score, Stromal_score, transcript_level_visium_barcode

💡 提示：你可以复制上面这一行，然后粘贴要删除的列（或部分列）

✅ 未删除任何列
✨ 清洗完成：保留 19 列。

🧩 检测到 13 个得分列：['B_cell_score', 'T_cell_score', 'DCIS_Tumor_score', 'Endothelial_score', 'Invasive_Tumor_score'] ...
✅ 分组平均完成: 23718 行
✅ 宽表完成: 3953 × 79
💾 保存结果：
  ├─ 细胞级 truth_score：CellType_GP/data/preprocessing_outputs/truth_output/truth_score.csv
  ├─ 分组均值 truth_result：CellType_GP/data/preprocessing_outputs/truth_output/truth_result_grouped.csv
  └─ 宽表 truth_result(wide)：CellType_GP/data/preprocessing_outputs/truth_outp

,spot,Endothelial+B_cell_score,Myeloid+B_cell_score,Myoepi+B_cell_score,Stromal+B_cell_score,Tumor+B_cell_score,lymphocyte+B_cell_score,Endothelial+DCIS_Tumor_score,Myeloid+DCIS_Tumor_score,Myoepi+DCIS_Tumor_score,...,Myoepi+Stromal_score,Stromal+Stromal_score,Tumor+Stromal_score,lymphocyte+Stromal_score,Endothelial+T_cell_score,Myeloid+T_cell_score,Myoepi+T_cell_score,Stromal+T_cell_score,Tumor+T_cell_score,lymphocyte+T_cell_score
0,AACACGTGCATCGCAC-1,NaN,0.057728,NaN,0.073599,NaN,0.549318,NaN,0.000000,NaN,...,NaN,0.337685,NaN,0.163545,NaN,0.115456,NaN,0.118845,NaN,0.101878
1,AACACTTGGCAAGGAA-1,0.128809,0.078966,0.065988,0.095118,0.043457,0.089914,0.163272,0.069038,0.263899,...,0.087460,0.279764,0.020771,0.161160,0.073331,0.092947,0.027743,0.079675,0.018356,0.377198
2,AACAGGAAGAGCATAG-1,NaN,NaN,NaN,0.099600,NaN,NaN,NaN,NaN,NaN,...,NaN,0.245806,NaN,NaN,NaN,NaN,NaN,0.055011,NaN,NaN
3,AACAGGATTCATAGTT-1,0.059409,0.123784,NaN,0.098263,NaN,0.208825,0.167652,0.031730,NaN,...,NaN,0.325493,NaN,0.223080,0.007399,0.109801,NaN,0.093613,NaN,0.393630
4,AACAGGTTATTGCACC-1,NaN,0.109359,NaN,0.099462,NaN,NaN,NaN,0.109130,NaN,...,NaN,0.399800,NaN,NaN,NaN,0.050699,NaN,0.088700,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,TGTTGGAACCTTCCGC-1,NaN,NaN,NaN,0.137039,NaN,NaN,NaN,NaN,NaN,...,NaN,0.346246,NaN,NaN,NaN,NaN,NaN,0.079326,NaN,NaN
3949,TGTTGGAACGAGGTCA-1,NaN,0.035474,NaN,0.056281,NaN,NaN,NaN,0.043660,NaN,...,NaN,0.325966,NaN,NaN,NaN,0.117204,NaN,0.096522,NaN,NaN
3950,TGTTGGAAGCTCGGTA-1,0.095399,0.107578,0.086432,0.115116,0.019499,0.155714,0.168419,0.071681,0.310125,...,0.072806,0.266911,0.054764,0.113345,0.154278,0.133230,0.046162,0.233029,0.009473,0.347972
3951,TGTTGGATGGACTTCT-1,0.059118,0.094249,NaN,0.055269,0.080250,0.123272,0.097153,0.073180,NaN,...,NaN,0.206140,0.040168,0.136599,0.051895,0.080963,NaN,0.075796,0.016829,0.313639


## 4. Visium 基因程序打分

对 Visium 空间数据重复基因程序打分，并保存处理后的 AnnData 文件。


In [5]:
# ---------------------------------------------------------------------
# 5. Visium 数据打分
# ---------------------------------------------------------------------
adata_v = sc.read(str(args.visium))
score_gene_programs(adata_v, gene_sets_to_score, platform="visium", output_dir=str(outdir / "visium_scores"))
adata_v.write(str(outdir / "vdata_processed.h5"))


🔹 正在处理: DCIS_1_score
✅ 完成: DCIS_1_score (20 基因)

🔹 正在处理: DCIS_2_score
✅ 完成: DCIS_2_score (20 基因)

🔹 正在处理: Prolif_Invasive_score
⚠️ 缺失基因 (Prolif_Invasive_score): POLR2J3
✅ 完成: Prolif_Invasive_score (19 基因)

🔹 正在处理: Invasive_Tumor_score
✅ 完成: Invasive_Tumor_score (20 基因)


🎉 所有基因集打分完成（使用平均表达），结果已保存至 CellType_GP/data/preprocessing_outputs/visium_scores/



## 5. 数据对齐与 NPZ 导出

同步 Visium 得分、空间坐标与细胞类型比例矩阵，最终生成 `spot_data_full.npz`。


In [ ]:
# ---------------------------------------------------------------------
# 6. 对齐 + 生成标准化 NPZ 文件
#    重新计算细胞比例（不再使用外部 fractions 文件），按 Xenium 的 broad_annotation 聚合
# ---------------------------------------------------------------------
# 使用 Xenium obs 中的条形码与大类注释，统计每个 Visium spot 的各大类细胞占比
x_obs = adata_x.obs[["transcript_level_visium_barcode", "broad_annotation"]].dropna(subset=["transcript_level_visium_barcode", "broad_annotation"])
spot_cluster_counts = (
    x_obs.groupby(["transcript_level_visium_barcode", "broad_annotation"])
    .size()
    .reset_index(name="cell_count")
)
spot_total_counts = (
    spot_cluster_counts.groupby("transcript_level_visium_barcode")["cell_count"]
    .sum()
    .reset_index(name="total_cells")
)
spot_cluster_stats = spot_cluster_counts.merge(spot_total_counts, on="transcript_level_visium_barcode")
spot_cluster_stats["fraction"] = spot_cluster_stats["cell_count"] / spot_cluster_stats["total_cells"]
fractions = spot_cluster_stats.pivot_table(
    index="transcript_level_visium_barcode",
    columns="broad_annotation",
    values="fraction",
    fill_value=0
)
fractions.index = fractions.index.astype(str)

# 对齐顺序（严格使用交集，避免 KeyError）/ strict intersection
target_spots = pd.Index(fractions.index.astype(str))
vis_spots = adata_v.obs_names.astype(str)
common = target_spots.intersection(vis_spots)
missing_in_visium = target_spots.difference(vis_spots)
missing_in_fraction = vis_spots.difference(target_spots)
if len(missing_in_visium):
    print(f"⚠️ 在 Visium 中缺失 {len(missing_in_visium)} 个 fractions 索引（已丢弃）：前3个示例：{list(missing_in_visium[:3])}")
if len(missing_in_fraction):
    print(f"⚠️ 在 fractions 中缺失 {len(missing_in_fraction)} 个 Visium 索引（已丢弃）：前3个示例：{list(missing_in_fraction[:3])}")

# 重排为相同顺序 / reorder both
fractions = fractions.loc[common]
adata_v = adata_v[common].copy()
print(f"✅ Visium 与 fractions 已对齐，共 {adata_v.n_obs} 个公共 spot。")

# 生成矩阵 / build matrices
# 程序列按定义顺序提取，保证与 gene_sets_to_score 对应；过滤不存在列
# desired_cols = [f"{k}_norm" for k in gene_sets_to_score.keys()] # 如果是未标准化，则注释掉
# score_cols = [c for c in desired_cols if c in adata_v.obs.columns]

score_cols = [c for c in gene_sets_to_score if c in adata_v.obs.columns]

if len(score_cols) == 0:
    raise RuntimeError("❌ 未在 Visium obs 中找到任何 *_score_norm 列，请先成功打分。")

visium_score = adata_v.obs[score_cols].values.T            # (P, S)
coords = adata_v.obsm["spatial"] if "spatial" in adata_v.obsm_keys() else np.zeros((adata_v.n_obs, 2))

spot_names = fractions.index.values.astype(str)
# 细胞类型名称从 fractions 列派生，确保顺序一致 / derive ct names from fractions columns
celltype_names = np.array(fractions.columns.values, dtype=str)
program_names = np.array(score_cols, dtype=str)

# 保存坐标 CSV（便于可视化）
coords_df = pd.DataFrame(coords, columns=["x", "y"])
coords_df.to_csv(outdir / "vdata_spatial_coords.csv", index=False)
print("✅ 已保存到", outdir / "vdata_spatial_coords.csv", "形状：", coords_df.shape)

# 保存 npz（标准输入包）
npz_path = outdir / "spot_data_full.npz"
np.savez_compressed(
    npz_path,
    spot_cluster_fraction_matrix=fractions.values.astype(np.float32), # 细胞比例（模拟反卷积结果）
    coords=coords.astype(np.float32), # 坐标
    visium_score=visium_score.astype(np.float32), # 待分解的低分辨率分数
    spot_names=spot_names, 
    celltype_names=celltype_names,
    program_names=program_names,
)
print("🎉 成功保存：", npz_path)

print("✅ 矩阵形状：")
print("  spot_cluster_fraction_matrix:", fractions.shape)
print("  visium_score:", visium_score.shape)
print("  coords:", coords.shape)


/tmp/ipykernel_3927106/1763122239.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  x_obs.groupby(["transcript_level_visium_barcode", "broad_annotation"])
/tmp/ipykernel_3927106/1763122239.py:19: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  fractions = spot_cluster_stats.pivot_table(


✅ Visium 与 fractions 已对齐，共 3953 个公共 spot。
✅ 已保存到 CellType_GP/data/preprocessing_outputs/vdata_spatial_coords.csv 形状： (3953, 2)
🎉 成功保存： CellType_GP/data/preprocessing_outputs/spot_data_full.npz
✅ 矩阵形状：
  spot_cluster_fraction_matrix: (3953, 6)
  visium_score: (13, 3953)
  coords: (3953, 2)


In [7]:
fractions= pd.read_csv('/home/vs_theg/ST_program/CellType_GP/data/spot_cluster_fraction_matrix.csv`')